<a href="https://colab.research.google.com/github/vihaanshah2014/jam/blob/main/echo_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Jun 11 14:44:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             45W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
!pip install --upgrade datasets transformers accelerate trl[sft]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from datasets import load_dataset

# 2.1 Load
tldr_ds = load_dataset("trl-lib/tldr")

# 2.2 Inspect splits and column names
print(tldr_ds)
# DatasetDict({
#     train: Dataset({ features: ['prompt','completion'], num_rows: … }),
#     validation: Dataset({ … }),
#     test: Dataset({ … })
# })

print(tldr_ds["train"].column_names)
# ['prompt', 'completion']

# 2.3 Peek at one example
print(tldr_ds["train"][0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/110M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/6.11M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/6.21M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/116722 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6447 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6553 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 116722
    })
    validation: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 6447
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 6553
    })
})
['prompt', 'completion']
{'prompt': "SUBREDDIT: r/relationships\n\nTITLE: I (f/22) have to figure out if I want to still know these girls or not and would hate to sound insulting\n\nPOST: Not sure if this belongs here but it's worth a try. \n\nBackstory:\nWhen I (f/22) went through my first real breakup 2 years ago because he needed space after a year of dating roand  it effected me more than I thought. It was a horrible time in my life due to living with my mother and finally having the chance to cut her out of my life. I can admit because of it was an emotional wreck and this guy was stable and didn't know how to deal with me. We ended by him avoiding for a month or so after going 

In [4]:
from datasets import load_dataset

tldr_ds   = load_dataset("trl-lib/tldr")
train_ds  = tldr_ds["train"]       # 116,722 examples
valid_ds  = tldr_ds["validation"]  #   6,447 examples
test_ds   = tldr_ds["test"]        #   6,553 examples

print(train_ds.column_names)  # ['prompt', 'completion']
print(train_ds.num_rows, valid_ds.num_rows, test_ds.num_rows)


['prompt', 'completion']
116722 6447 6553


In [12]:
from transformers import AutoTokenizer

model_name = "google/flan-t5-base"
tokenizer  = AutoTokenizer.from_pretrained(model_name)

def tokenize_batch(batch):
    # inputs
    tok = tokenizer(
        batch["prompt"],
        max_length=512,
        truncation=True,
        padding="max_length"
    )
    # targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["completion"],
            max_length=128,
            truncation=True,
            padding="max_length"
        )
    tok["labels"] = labels["input_ids"]
    return tok

# Use batched=True so this streams and doesn’t error
train_tok = train_ds.map(
    tokenize_batch,
    batched=True,
    batch_size=1000,
    remove_columns=train_ds.column_names
)
valid_tok = valid_ds.map(
    tokenize_batch,
    batched=True,
    batch_size=1000,
    remove_columns=valid_ds.column_names
)
test_tok = test_ds.map(
    tokenize_batch,
    batched=True,
    batch_size=1000,
    remove_columns=test_ds.column_names
)

Map:   0%|          | 0/6447 [00:00<?, ? examples/s]

In [20]:
from datasets import load_dataset

# Load the Codetotext split you want
code_ds = load_dataset("vietanave/Codetotext")["train"]

# Inspect to confirm…
print(code_ds.column_names)   # ['code', 'text']
print(code_ds[0])             # peek at one example

# Remap into prompt/completion
def preprocess_codetext(example):
    return {
        "prompt": example["code"],
        "completion": example["text"]
    }

# Apply
code_prep = code_ds.map(
    preprocess_codetext,
    batched=False,
    remove_columns=code_ds.column_names
)

# Verify
print(code_prep.column_names, code_prep.num_rows)
print(code_prep[0])


Repo card metadata block was not found. Setting CardData to empty.


['code', 'text']
{'code': '# Python 3 program to find \n# factorial of given number\ndef factorial(n):\n \n # single line to find factorial\n return 1 if (n==1 or n==0) else n * factorial(n - 1) \n\n# Driver Code\nnum = 5\nprint("Factorial of",num,"is",factorial(num))', 'text': 'This program is responsible for converting kilometers to miles'}


Map:   0%|          | 0/7654 [00:00<?, ? examples/s]

['prompt', 'completion'] 7654
{'prompt': '# Python 3 program to find \n# factorial of given number\ndef factorial(n):\n \n # single line to find factorial\n return 1 if (n==1 or n==0) else n * factorial(n - 1) \n\n# Driver Code\nnum = 5\nprint("Factorial of",num,"is",factorial(num))', 'completion': 'This program is responsible for converting kilometers to miles'}


In [21]:
explain_ds = load_dataset("anujbishtTx/code_snippets_explaination")
print(explain_ds)
print(explain_ds["train"].column_names)

def preprocess_explanation(example):
    src = example.get("sentences") or example.get("snippets") or ""
    return {"prompt": src, "completion": example["Explanation"]}

explain_prep = explain_ds["train"].map(
    preprocess_explanation,
    remove_columns=explain_ds["train"].column_names
)

explain_tok = explain_prep.map(
    tokenize_batch,
    batched=True,
    batch_size=1000,
    remove_columns=explain_prep.column_names
)

README.md:   0%|          | 0.00/413 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/400k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/108k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/716 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/180 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentences', 'Explanation'],
        num_rows: 716
    })
    test: Dataset({
        features: ['sentences', 'Explanation'],
        num_rows: 180
    })
})
['sentences', 'Explanation']


Map:   0%|          | 0/716 [00:00<?, ? examples/s]

Map:   0%|          | 0/716 [00:00<?, ? examples/s]

In [22]:
n_ex = explain_tok.num_rows
test_size = int(0.1 * n_ex)
explain_test_tok = explain_tok.select(range(test_size))
explain_train_tok = explain_tok.select(range(test_size, n_ex))

from datasets import concatenate_datasets
full_train = concatenate_datasets([train_tok, code_tok, explain_train_tok])
print("Combined training size:", full_train.num_rows)

Combined training size: 117437


In [23]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = AutoModelForSeq2SeqLM.from_pretrained(model_name).cuda()

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [24]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="tuned_tldr",
    per_device_train_batch_size=8,    # ↑ from 4
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,    # ↓ from 4
    num_train_epochs=3,
    learning_rate=3e-5,
    bf16=True,                        # switched from fp16
    logging_steps=100,
    save_total_limit=2,
    dataloader_num_workers=4,
)

In [25]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=valid_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# 4.1 Fine-tune
trainer.train()

# 4.2 Manual evaluation
print("Validation (TLDR):", trainer.evaluate(valid_tok))
print("Test (Code explanation):", trainer.evaluate(explain_test_tok))

<ipython-input-25-3304858905>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vihaan-shah2014 (vihaan-shah2014-rutgers-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
100,9.615000
200,1.080400
300,0.758800
400,0.746200
500,0.729400
600,0.732300
700,0.719300
800,0.716200
900,0.732900
1000,0.723300


Validation (TLDR): {'eval_loss': 0.6332440972328186, 'eval_runtime': 41.1142, 'eval_samples_per_second': 156.807, 'eval_steps_per_second': 19.604, 'epoch': 3.0}
Test (Code explanation): {'eval_loss': 3.041317939758301, 'eval_runtime': 0.8238, 'eval_samples_per_second': 86.188, 'eval_steps_per_second': 10.925, 'epoch': 3.0}


In [26]:
trainer.save_model("tuned_tldr/final")
tokenizer.save_pretrained("tuned_tldr/final")

('tuned_tldr/final/tokenizer_config.json',
 'tuned_tldr/final/special_tokens_map.json',
 'tuned_tldr/final/spiece.model',
 'tuned_tldr/final/added_tokens.json',
 'tuned_tldr/final/tokenizer.json')

In [27]:
# 1. Create a clean folder (optional if it doesn’t already exist)
import os
os.makedirs("saved_model", exist_ok=True)

# 2. Save the model weights + config
trainer.save_model("saved_model")       # writes pytorch_model.bin, config.json, etc.

# 3. Save the tokenizer files
tokenizer.save_pretrained("saved_model")  # writes vocab, merges (if BPE), tokenizer_config.json, etc.

('saved_model/tokenizer_config.json',
 'saved_model/special_tokens_map.json',
 'saved_model/spiece.model',
 'saved_model/added_tokens.json',
 'saved_model/tokenizer.json')

In [28]:
# 1. Re-save to ensure folder is up-to-date
trainer.save_model("saved_model")
tokenizer.save_pretrained("saved_model")

# 2. Create a ZIP archive of the folder
!zip -r saved_model.zip saved_model

# 3. Download the ZIP to your local machine
from google.colab import files
files.download("saved_model.zip")

  adding: saved_model/ (stored 0%)
  adding: saved_model/training_args.bin (deflated 52%)
  adding: saved_model/config.json (deflated 62%)
  adding: saved_model/generation_config.json (deflated 29%)
  adding: saved_model/tokenizer.json (deflated 74%)
  adding: saved_model/model.safetensors (deflated 7%)
  adding: saved_model/special_tokens_map.json (deflated 85%)
  adding: saved_model/tokenizer_config.json (deflated 95%)
  adding: saved_model/spiece.model (deflated 48%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>